# Higher-Order Functions

We have seen that functions are, in effect, abstractions that describe compound operations on numbers independent of the particular numbers. 

For example, when we declare

In [ ]:
def cube(x):
    return x * x * x

we are not talking about the cube of a particular number, but rather about a method for obtaining the cube of any number. 

Of course we could get along without ever declaring this function, by always writing expressions such as

$ 3*3*3$ \
$x*x*x $ \
$ y*y*y $

and never mentioning cube explicitly. 

This would place us at a serious disadvantage, forcing us to work always at the level of the particular operations that happen to be primitives in the language (multiplication, in this case) rather than in terms of higher-level operations. 

Our programs would be able to compute cubes, but our language would lack the ability to express the concept of cubing. One of the things we should demand from a powerful programming language is the ability to build abstractions by assigning names to common patterns and then to work in terms of the abstractions directly. Functions provide this ability. This is why all but the most primitive programming languages include mechanisms for declaring functions.

Yet even in numerical processing we will be severely limited in our ability to create abstractions if we are restricted to functions whose parameters must be numbers. Often the same programming pattern will be used with a number of different functions. To express such patterns as concepts, we will need to construct functions that can accept functions as arguments or return functions as values. 

Functions that manipulate functions are called **higher-order functions**. 

This section shows how higher-order functions can serve as powerful abstraction mechanisms, vastly increasing the expressive power of our language.

## Functions as Arguments

Consider the following three functions: 

1. The first computes the sum of the integers from a through b: 

```python
def sum_integers(a, b):
    if a > b:
        return 0
    else:
        return a + sum_integers(a + 1, b)
```

2. The second computes the sum of the cubes of the integers in the given range:

```python
def sum_cubes(a, b):
    if a > b:
        return 0
    else:
        return cube(a) + sum_cubes(a + 1, b)
```

3. The third computes the sum of a sequence of terms in a series

$$ 1 + \frac{1}{3} + \frac{1}{5} + \frac{1}{7} + \cdots + \frac{1}{n} $$


1+1+ 1 +···
 1·3 5·7 9·11
which converges to π/8 (very slowly):49

, where the nth term of the series is defined to be $1/n(n+1)$:

```python

def pi_sum(a, b):
    if a > b:
        return 0
    else:
        return 1.0 / (a * (a + 2)) + pi_sum(a + 4, b)
```

These three functions clearly share a common underlying pattern. They are for the most part identical, differing only in the name of the function, the function of a used to compute the term to be added, and the function that provides the next value of a. We could generate each of the functions by filling in slots in the same template:

```python
def <name>(a, b):
    if a > b:
        return 0
    else:
        return <term>(a) + <name>(<next>(a), b)
```

where the placeholders <name>, <term>, and <next> are to be replaced by the name of the function, the function of a used to compute the term to be added, and the function that provides the next value of a, respectively.

The presence of such a common pattern is strong evidence that there is a useful abstraction waiting to be brought to the surface. Indeed, mathematicians long ago identified the abstraction of summation of a series and invented “sigma notation,” for example

$$ \sum^{b}_{n=a} f(n) = f(a) + \cdots + f(b) $$

to express this concept. The power of sigma notation is that it allows mathemati- cians to deal with the concept of summation itself rather than only with particular sums—for example, to formulate general results about sums that are independent of the particular series being summed.
Similarly, as program designers, we would like our language to be powerful enough so that we can write a function that expresses the concept of summation itself rather than only functions that compute particular sums. We can do so readily in our functional language by taking the common template shown above and transforming the “slots” into parameters:

In [ ]:
def sum(term, a, next, b):
    if a > b:
        return 0
    else:
        return term(a) + sum(term, next(a), next, b)

Notice that sum takes as its arguments the lower and upper bounds a and b together with the functions term and next. We can use sum just as we would any function. For example, we can use it (along with a function inc that increments its argument by 1) to define sum_cubes:

In [ ]:
def inc(n):
    return n + 1

def sum_cubes(a, b):
    return sum(cube, a, inc, b)

Using this, we can compute the sum of the cubes of the integers from 1 to 10:

In [ ]:
sum_cubes(1, 10)

With the aid of an identity function to compute the term, we can define sum_ integers in terms of sum:

In [ ]:
def identity(x):
    return x

def sum_integers(a, b):
    return sum(identity, a, inc, b)

Then we can add up the integers from 1 to 10:

In [ ]:
sum_integers(1, 10)

We can also define pi_sum in the same way:

In [ ]:

def pi_sum(a, b):
    def pi_term(x):
        return 1 / (x * (x + 2))
    def pi_next(x):
        return x + 4
    return sum(pi_term, a, pi_next, b)

Using these functions, we can compute an approximation to π:

In [ ]:
8 * pi_sum(1, 1000)

Once we have sum, we can use it as a building block in formulating further concepts. For instance, the definite integral of a function f between the limits a and b can be approximated numerically using the formula

$$ \int^{b}_{a} f(x)dx \approx \sum^{n}_{i=1} f(a + \frac{i - 1}{n}(b - a)) \frac{b - a}{n} $$

where n is the number of steps to use in the approximation. We can express this directly as a function:

In [ ]:
def integral(f, a, b, dx):
    def add_dx(x):
        return x + dx
    return sum(f, a + dx / 2, add_dx, b) * dx

In [ ]:
integral(cube, 0, 1, 0.01)

In [ ]:
integral(cube, 0, 1, 0.001)

(The exact value of the integral of cube between 0 and 1 is 1/4.)

## Constructing Functions using Lambda Expressions

In using sum as in section 1.3.1, it seems terribly awkward to have to declare trivial functions such as pi_term and pi_next just so we can use them as arguments to our higher-order function. Rather than declare pi_next and pi_term, it would be more convenient to have a way to directly specify “the function that returns its input incremented by 4” and “the function that returns the reciprocal of its input times its input plus 2.” We can do this by introducing the lambda expression as a syntactic form for creating functions. Using lambda expressions, we can describe what we want as

$ x \Rightarrow x + 4 $ 

```python
lambda x: x + 4
```

and 

$ x \Rightarrow \frac{1}{(x(x + 2))} $


```python
lambda x: 1 / (x * (x + 2))
```


Then we can express our pi_sum function without declaring any auxiliary functions:

In [ ]:
def pi_sum(a, b):
    return sum(lambda x: 1 / (x * (x + 2)),
               a,
               lambda x: x + 4,
               b)

Again using a lambda expression, we can write the integral function without having to declare the auxiliary function `add_dx`:

In [ ]:
def integral(f, a, b, dx):
    return sum(f, a + dx / 2, lambda x: x + dx, b) * dx

In general, lambda expressions are used to create functions in the same way as function declarations, except that no name is specified for the function and the return keyword and braces are omitted (if there is only one parameter, the paren- theses around the parameter list can also be omitted, as in the examples we have seen).

$$ (parameters) => expression $$

The resulting function is unnamed, but we can bind it to a name using an assignment statement, just as we can bind the value of any expression to a name. For example, we can define the function cube using a lambda expression:

```python
lambda parameters: expression
```

The resulting function is just as much a function as one that is created using a function declaration statement. The only difference is that it has not been associated with any name in the environment. We consider

In [ ]:
def plus4(x):
    return x + 4

to be equivalent to

In [ ]:
plus4 = lambda x: x + 4

We can read a lambda expression as follows:

``` python
lambda x  # function of an argument x
:         # that results in 
x         # the value 
+         # plus
4         # 4
```

Like any expression that has a function as its value, a lambda expression can be used as the function expression in an application such as

In [1]:
square = lambda x: x * x

(lambda x, y, z: x + y + square(z))(1, 2, 3)

12

or, more generally, in any context where we would normally use a function name.55 Note that => has lower precedence than function application and thus the parentheses around the lambda expression are necessary here.

## Functions as General Methods

We introduced compound functions in section 1.1.4 as a mechanism for abstracting patterns of numerical operations so as to make them independent of the particular numbers involved. With higher-order functions, such as the integral function of section 1.3.1, we began to see a more powerful kind of abstraction: functions used to express general methods of computation, independent of the particular functions involved. In this section we discuss two more elaborate examples—general methods for finding zeros and fixed points of functions—and show how these methods can be expressed directly as functions.

### Finding roots of equations by the half-interval method

The half-interval method is a simple but powerful technique for finding roots of an equation $f (x) = 0$, where $f$ is a continuous function. 

The idea is that, if we are given points $a$ and $b$ such that $f(a)<0<f(b)$, then $f$ must have at least one zero between a and b. 

To locate a zero, let $x$ be the average of a and b and compute $f(x)$. 

If $f(x)>0$, then f must have a zero between a and x. 

If $f (x) < 0$, then $f$ must have a zero between $x$ and $b$. 

Continuing in this way, we can identify smaller and smaller intervals on which f must have a zero. 

When we reach a point where the interval is small enough, the process stops. 

Since the interval of uncertainty is reduced by half at each step of the process, the maximal number of steps required grows as $\Theta(\text{log}(L/T))$, where $L$ is the length of the original interval and $T$ is the error tolerance (that is, the size of the interval we will consider “small enough”). Here is a function that implements this strategy:

In [ ]:
def close_enough(x, y, tolerance=0.001):
    return abs(x - y) < tolerance

def search(f, neg_point, pos_point):
    midpoint = (neg_point + pos_point) / 2
    if close_enough(neg_point, pos_point):
        return midpoint
    test_value = f(midpoint)
    if test_value > 0:
        return search(f, neg_point, midpoint)
    elif test_value < 0:
        return search(f, midpoint, pos_point)
    else:
        return midpoint

We assume that we are initially given the function f together with points at which its values are negative and positive. We first compute the midpoint of the two given points. Next we check to see if the given interval is small enough, and if so we simply return the midpoint as our answer. Otherwise, we compute as a test value the value of f at the midpoint. If the test value is positive, then we continue the process with a new interval running from the original negative point to the midpoint. If the test value is negative, we continue with the interval from the midpoint to the positive point. Finally, there is the possibility that the test value is 0, in which case the midpoint is itself the root we are searching for. To test whether the endpoints are “close enough” we can use a function similar to the one used in section 1.1.7 for computing square roots:

The function search is awkward to use directly, because we can accidentally give it points at which f ’s values do not have the required sign, in which case we get a wrong answer. Instead we will use search via the following function, which checks to see which of the endpoints has a negative function value and which has a positive value, and calls the search function accordingly. If the function has the same sign on the two given points, the half-interval method cannot be used, in which case the function signals an error.

In [ ]:
def half_interval_method(f, a, b):
    a_value = f(a)
    b_value = f(b)
    if a_value < 0 and b_value > 0:
        return search(f, a, b)
    elif b_value < 0 and a_value > 0:
        return search(f, b, a)
    else:
        print("Values are not of opposite sign")

The following example uses the half-interval method to approximate π as the root between 2 and 4 of sin x=0:


In [ ]:
from math import sin 

half_interval_method(sin, 2, 4)

Here is another example, using the half-interval method to search for a root of the equation $x^3 –2x–3=0$ between 1 and 2:

In [ ]:
half_interval_method(lambda x : x * x * x - 2 * x - 3, 1, 2)

### Fixed points of functions

A number $x$ is called a _fixed point_ of a function $f$ if $x$ satisfies the equation $f (x) = x$. 

For some functions f we can locate a fixed point by beginning with an initial guess and applying f repeatedly,

$$ f(x), f(f(x)), f(f(f(x))), \cdots $$

until the value does not change very much.

Using this idea, we can devise a function `fixed_point` that takes as inputs a function and an initial guess and produces an approximation to a fixed 
point of the function. We apply the function repeatedly until we find two successive values whose difference is less than some prescribed tolerance:


In [ ]:


def fixed_point(f, first_guess, tolerance=0.00001):

    def close_enough(v1, v2):
        return abs(v1 - v2) < tolerance
    
    def try_with(guess):
        next_ = f(guess)
        if close_enough(guess, next_):
            return next_
        else:
            return try_with(next_)
    
    return try_with(first_guess)



For example, we can use this method to approximate the fixed point of the cosine function, starting with 1 as an initial approximation

In [ ]:
from math import sin, cos
fixed_point(cos, 1)

Similarly, we can find a solution to the equation $y = sin y + cos y$:

In [ ]:
from math import sin, cos
fixed_point(lambda y :sin(y) + cos(y), 1)

The fixed-point process is reminiscent of the process we used for finding square roots in section 1.1.7. Both are based on the idea of repeatedly improving a guess until the result satisfies some criterion. In fact, we can readily formulate the square- root computation as a fixed-point search. Computing the square root of some number x requires finding a y such that y2 = x. Putting this equation into the equivalent form $y = x/y$, we recognize that we are looking for a fixed point of the function $y \mapsto x/y$, and we can therefore try to compute square roots as

In [ ]:
def sqrt(x):
    return fixed_point(lambda y: (y + x / y) / 2, 1)

Unfortunately, this fixed-point search does not converge. Consider an initial guess $y_1$. The next guess is $y_2 = x/y_1$ and the next guess is $y3 = x/y_2 = x/(x/y_1) = y_1$. This results in an infinite loop in which the two guesses $y_1$ and $y_2$ repeat over and over, oscillating about the answer.

One way to control such oscillations is to prevent the guesses from changing so much. Since the answer is always between our guess y and x/y, we can make a new guess that is not as far from y as x/y by averaging y with $x/y$, so that the next guess after y is $\frac{1}{2} (y + \frac{x}{y})$ instead of x/y. The process of making such a sequence of guesses is simply the process of looking for a fixed point of $y \mapsto 12 (y + x/y)$:

In [ ]:
def average(x, y):
    return (x + y) / 2

def sqrt(x):
    return fixed_point(lambda y: average(y, x / y), 1)

(Note that $y = \frac{1}{2} (y + \frac{x}{y})$ is a simple transformation of the equation $y = \frac{x}{y}$; to derive it, add $y$ to both sides of the equation and divide by $2$.)

With this modification, the square-root function works. In fact, if we unravel the definitions, we can see that the sequence of approximations to the square root generated here is precisely the same as the one generated by our original square-root function of section 1.1.7. This approach of averaging successive approximations to a solution, a technique we call _average damping_, often aids the convergence of fixed-point searches.

## Functions as Returned Values

The above examples demonstrate how the ability to pass functions as arguments significantly enhances the expressive power of our programming language. We can achieve even more expressive power by creating functions whose returned values are themselves functions.

We can illustrate this idea by looking again at the fixed-point example described at the end of section 1.3.3. We formulated a new version of the square-root function as a fixed-point search, starting with the observation that $x$ is a fixed-point of the function $y \mapsto x/y$. Then we used average damping to make the approximations converge. Average damping is a useful general technique in itself. Namely, given a function $f$ , we consider the function whose value at $x$ is equal to the average of $x$ and $f(x)$.

We can express the idea of average damping by means of the following function:

In [ ]:
def average_damp(f):
    return lambda x: average(x, f(x))

The function average_damp takes as its argument a function $f$ and returns as its value a function (produced by the lambda expression) that, when applied to a number $x$, produces the average of $x$ and $f(x)$. For example, applying average_damp to the square function produces a function whose value at some number $x$ is the average of $x$ and $x^2$. Applying this resulting function to 10 returns the average of 10 and 100, or 55:

In [ ]:
average_damp(square)(10)

Using `average_damp`, we can reformulate the square-root function as follows:

In [ ]:
def sqrt(x):
    return fixed_point(average_damp(lambda y: x / y), 1)

Notice how this formulation makes explicit the three ideas in the method: fixed- point search, average damping, and the function $y \mapsto x/y$. It is instructive to compare this formulation of the square-root method with the original version given in section 1.1.7. Bear in mind that these functions express the same process, and notice how much clearer the idea becomes when we express the process in terms of these abstractions. In general, there are many ways to formulate a process as a function. Experienced programmers know how to choose process formulations that are partic- ularly perspicuous, and where useful elements of the process are exposed as separate entities that can be reused in other applications. As a simple example of reuse, notice that the cube root of $x$ is a fixed point of the function $y \mapsto x/y_2$, so we can immediately generalize our square-root function to one that extracts cube roots:

In [ ]:
def cube_root(x):
    return fixed_point(average_damp(lambda y: x / (y * y)), 1)

#### Newton's method 

When we first introduced the square-root function, in section 1.1.7, we mentioned that this was a special case of Newton’s method. If $x \mapsto g(x)$ is a differentiable function, then a solution of the equation $g(x) = 0$ is a fixed point of the function $x \mapsto f (x)$ where

$$ f(x) = x - \frac{g(x)}{Dg(x)} $$


and Dg(x) is the derivative of g evaluated at x. Newton’s method is the use of the fixed-point method we saw above to approximate a solution of the equation by finding a fixed point of the function f .64 For many functions g and for sufficiently good initial guesses for x, Newton’s method converges very rapidly to a solution of g(x) = 0.

In order to implement Newton’s method as a function, we must first express the idea of derivative. Note that “derivative,” like average damping, is something that transforms a function into another function. For instance, the derivative of the function $x \mapsto x^3$ is the function $x \mapsto 3x^2$. In general, if g is a function and dx is a small number, then the derivative $Dg$ of $g$ is the function whose value at any number x is given (in the limit of small dx) by

$$ Dg(x) = \frac{g(x + dx) - g(x)}{dx} $$

Thus, we can express the idea of derivative (taking dx to be, say, 0.00001) as the function

In [ ]:
def deriv(g):
    dx = 0.0000001
    return lambda x: (g(x + dx) - g(x)) / dx

Like average_damp, deriv is a function that takes a function as argument and returns a function as value. For example, to approximate the derivative of $x \mapsto x3$ at 5 (whose exact value is 75) we can evaluate

In [ ]:
def cube(x):
    return x * x * x

In [ ]:
deriv(cube)(5)

With the aid of deriv, we can express Newton’s method as a fixed-point process:

In [ ]:
def newton_transform(g):
    return lambda x: x - g(x) / deriv(g)(x)

def newtons_method(g, guess):
    return fixed_point(newton_transform(g), guess)

The newton_transform function expresses the formula at the beginning of this sec- tion, and newtons_method is readily defined in terms of this. It takes as arguments a function that computes the function for which we want to find a zero, together with an initial guess. For instance, to find the square root of x, we can use Newton’s method to find a zero of the function y 􏰂→ y2 – x starting with an initial guess of 1.66 This provides yet another form of the square-root function:

In [ ]:
def sqrt(x):
    return newtons_method(lambda y: y * y - x, 1)

#### Abstractions and first-class functions

We’ve seen two ways to express the square-root computation as an instance of a more general method, once as a fixed-point search and once using Newton’s method. 

Since Newton’s method was itself expressed as a fixed-point process, we actually saw two ways to compute square roots as fixed points. 

Each method begins with a function and finds a fixed point of some transformation of the function. 

We can express this general idea itself as a function:

In [ ]:
def fixed_point_of_transform(g, transform, guess):
    return fixed_point(transform(g), guess)

This very general function takes as its arguments a function $g$ that computes some function, a function that transforms $g$, and an initial guess. The returned result is a fixed point of the transformed function.

Using this abstraction, we can recast the first square-root computation from this section (where we look for a fixed point of the average-damped version of $y \mapsto \frac{x}{y}$) as an instance of this general method:

In [ ]:
def sqrt(x):
    return fixed_point_of_transform(lambda y: x / y, average_damp, 1)

Similarly, we can express the second square-root computation from this section (an instance of Newton’s method that finds a fixed point of the Newton transform of $y\mapstoy^2 –x$) as

In [ ]:
def sqrt(x):
    return fixed_point_of_transform(lambda y: square(y) - x, newton_transform, 1)

We began section 1.3 with the observation that compound functions are a cru- cial abstraction mechanism, because they permit us to express general methods of computing as explicit elements in our programming language. Now we’ve seen how higher-order functions permit us to manipulate these general methods to create further abstractions.

As programmers, we should be alert to opportunities to identify the underlying abstractions in our programs and to build upon them and generalize them to create more powerful abstractions. This is not to say that one should always write programs in the most abstract way possible; expert programmers know how to choose the level of abstraction appropriate to their task. But it is important to be able to think in terms of these abstractions, so that we can be ready to apply them in new contexts. The significance of higher-order functions is that they enable us to represent these abstractions explicitly as elements in our programming language, so that they can be handled just like other computational elements.

In general, programming languages impose restrictions on the ways in which computational elements can be manipulated. Elements with the fewest restrictions are said to have first-class status. Some of the “rights and privileges” of first-class elements are

* They may be referred to using names.
* They may be passed as arguments to functions.
* They may be returned as the results of functions. 
* They may be included in data structures

Python, like other high-level programming languages, awards functions full first-class status. 

This poses challenges for efficient implementation, but the resulting gain in expressive power is enormous.